# Start

In [1]:
import pandas as pd
import numpy as np
import pickle 

import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [1]:
# maximum text
# sb.set()
# pd.set_option('display.max_colwidth', None)

# Load dataset

## GabHateCorpus

In [9]:
dataset_name = "GabHateCorpus"
filepath = "Dataset/GabHateCorpus/"
df = pd.read_csv(filepath+"data_processed.csv")
df['class'].value_counts(normalize=True)

class
Non-Hate    0.876664
Hate        0.123336
Name: proportion, dtype: float64

## SE2019 dataset

In [2]:
dataset_name = "SE2019"
filepath = "Dataset/SE2019/"
df = pd.read_csv(filepath+"data_processed.csv")
df['class'].value_counts(normalize=True)

class
Non-Hate    0.578542
Hate        0.421458
Name: proportion, dtype: float64

## Implicit_hate_corpus

In [2]:
dataset_name = "Implicit_hate_corpus"
filepath = "Dataset/Implicit_hate_corpus/"
df = pd.read_csv(filepath+"data_processed.csv")
df['class'].value_counts(normalize=True)

class
Non-Hate    0.618726
Hate        0.381274
Name: proportion, dtype: float64

## data analyze

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21478 entries, 0 to 21477
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   21478 non-null  object
 1   text    21478 non-null  object
 2   hate    21478 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 503.5+ KB


In [41]:
df.head(5)

,class,text,hate
0,Hate,jewish harvard professor noel ignatiev wants ...,1
1,Non-Hate,bhigher education part european culture impor...,0
2,Non-Hate,problem whites christians go ahead free say,0
3,Non-Hate,yasir qadhi hate preacher calling christians ...,0
4,Non-Hate,rt three million germans mass murdered destru...,0


In [42]:
df[df['text'].isnull()]

,class,text,hate


In [41]:
filepath = "Dataset/SE2019/"
df_need = pd.read_csv(filepath+"data_needed.csv")
df_processed = pd.read_csv(filepath+"data_processed.csv")
print(len(df_need), len(df_processed))

13000 12974


In [42]:
text_need = np.array(df_need["text"])
text_processed = np.array(df_processed["text"])

In [37]:
def count_words(text):
    counter = 0
    for i in text:
        counter += len(i.split())
    return counter

def count_token(text):
    s = set()
    for i in text:
        tokenize = i.split()
        for j in tokenize:
            s.add(j)    
    return len(s)

In [43]:
print("word")
print("need: ", count_words(text_need))
print("pro: ", count_words(text_processed))

print("token")
print("need: ", count_token(text_need))
print("pro: ", count_token(text_processed))

word
need:  281058
pro:  156009
token
need:  53046
pro:  22855


# Split dataset

change test size to 10 fold cross validation    
so test size is 0.10 

In [27]:
test_size = 0.10
x = np.array(df["text"])
y = np.array(df["class"])

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = test_size, random_state=42) #random state ensure same sample
print("Train Set :", x_train.shape, y_train.shape) 
print("Test Set  :", x_test.shape, y_test.shape) 

# y in digit form
y_train_binary = np.array(list(map(lambda x:1 if x=="Hate" else 0, y_train)))
y_test_binary = np.array(list(map(lambda x:1 if x=="Hate" else 0, y_test)))

Train Set : (24401,) (24401,)
Test Set  : (2712,) (2712,)


0.30
- se2019 = orange colored scumpig cunt whore god soon go history worst potus brainwas ', <br>
- gab = mainstream muslims murder gays ', <br>
- implicit = 'would think sweden cops able use snipers shoot flaming bombs hands till crush muslims never stop muslims understand force',

10 fold
- gab = 'anyone help older samsung tablet get white screen trying log gab download says parsing error bookmarked still wont open even online browser help'


In [28]:
x_train[0]

'anyone help older samsung tablet get white screen trying log gab download says parsing error bookmarked still wont open even online browser help'

# Feature engineering - Word embeding

- Word2Vec and GloVe are based on static word embeddings while Transformers are based on dynamic word embeddings.

- Many different models for creating pre-trained word embeddings such as Word2Vec, GloVe, fastText, and BioWordVec 

- https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
- CountVectorizer, Tfidftransformer & Tfidfvectorizer are Frequency based Word Embedding technique
- Tfidftransformer acts on sparse matrix and Tfidfvectorizer acts on raw text data
- Tfidfvectorizer = countVectorizater + Tfidftransformer

- https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/
- vectorizer = word embedding process of converting text data to numerical vector

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Word2Vec

- https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/#:~:text=Word2Vec%20is%20a%20popular%20algorithm,a%20large%20corpus%20of%20text
- Word2vec is not a single algorithm but a combination of two techniques – CBOW(Continuous bag of words) and Skip-gram model.

In [9]:
from gensim.models import Word2Vec

In [10]:
class w2vVectorizer():
    def __init__(self) -> None:
        self.w2v_model = None
    
    def w2v_vectorizer(self,sentence):
        # vectorize the text data
        words = sentence.split()
        words_vec = [self.w2v_model.wv[word] for word in words if word in self.w2v_model.wv]
        if len(words_vec) == 0:
            return np.zeros(100)
        words_vec = np.array(words_vec)
        return words_vec.mean(axis=0)
    
    def fit(self, x, y=None):
        # train the model when fit the pipeline
        sentences = [sentence.split() for sentence in x]
        self.w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
        return self
    
    def transform(self, x, y=None):
        # when use fit or transform on the pipeline 
        return np.array([self.w2v_vectorizer(sentence) for sentence in x])

# Machine learning

In [10]:
# save model
def save_model(model, model_name):
    filename = f"models/{model_name}.pickle"
    pickle.dump(model, open(filename,"wb"))

## Decision Tree

In [14]:
#convert given text to a vector base
from sklearn.tree import DecisionTreeClassifier

In [424]:
model = Pipeline([('vect', CountVectorizer()),
               ('clf', DecisionTreeClassifier()),
              ])
model_name = "dtc"
model.fit(x_train, y_train)
save_model(model,model_name)

In [232]:
model = Pipeline([('vect', TfidfVectorizer()),
               ('clf', DecisionTreeClassifier()),
              ])
model_name = "dtc-tfid"
model.fit(x_train, y_train)
save_model(model,model_name)

In [233]:
model = Pipeline([('vect', w2vVectorizer()),
               ('clf', DecisionTreeClassifier()),
              ])
model_name = "dtc-w2v"
model.fit(x_train, y_train)
save_model(model,model_name)

## Support Vector Machine

- supervisied learning algorithm
- Unlike neural networks, SVMs can work with very small datasets and are not prone to overfitting.

In [39]:
from sklearn.linear_model import SGDClassifier

In [234]:
model = Pipeline([('vect', CountVectorizer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
model_name = "svm"
model.fit(x_train, y_train)
save_model(model, model_name)

In [235]:
model = Pipeline([('vect', TfidfVectorizer()),
               ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
              ])
model_name = "svm-tfid"
model.fit(x_train, y_train)
save_model(model, model_name)

In [236]:
model = Pipeline([('vect', w2vVectorizer()),
               ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
              ])
model_name = "svm-w2v"
model.fit(x_train, y_train)
save_model(model, model_name)

## Logistic Regression 

In [46]:
from sklearn.linear_model import LogisticRegression

In [237]:
model = Pipeline([('vect', CountVectorizer()),
        ('clf', LogisticRegression(n_jobs=1, C=1e5,max_iter=6300)),
        ])
model_name = "lr"
model.fit(x_train, y_train)
save_model(model, model_name)

In [238]:
model = Pipeline([('vect', TfidfVectorizer()),
        ('clf', LogisticRegression(n_jobs=1, C=1e5,max_iter=6300)),
        ])
model_name = "lr-tfid"
model.fit(x_train, y_train)
save_model(model, model_name)

In [239]:
model = Pipeline([('vect', w2vVectorizer()),
        ('clf', LogisticRegression(n_jobs=1, C=1e5,max_iter=6300)),
        ])
model_name = "lr-w2v"
model.fit(x_train, y_train)
save_model(model, model_name)

## Predict

In [ ]:
# Predict data
print("Test Data Accuracy  :\t", model.score(x_test, y_test))
y_test_pred = model.predict(x_test)

# Deep Neural network

tensorflow gpu 
- https://www.tensorflow.org/install/pip#windows-native
- https://lifewithdata.com/2022/01/16/how-to-install-tensorflow-and-keras-with-gpu-support-on-windows/ 

- https://www.youtube.com/watch?v=oWo9SNcyxlI
- good read for = https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-022-01665-y

- cnn
- transformer encoder
- pretrained bert
- typical neural network = rnn, gru, lstm, bi-lstm

- other than bert, the rest need word embedding so do glove, word2vec and fasttext


In [4]:
from keras.models import Sequential
from keras.utils.data_utils import pad_sequences
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding
from keras.layers import Conv1D, LSTM, SpatialDropout1D, Bidirectional, GRU, SimpleRNN, TextVectorization

from keras.metrics import BinaryAccuracy,Precision,Recall
import keras
from keras.models import load_model
import tensorflow as tf

from gensim.models import FastText, Word2Vec

METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall")
]

def save_model_nn(model, model_name, embedding_name, dataset_name):
    filename = f"models/{dataset_name}_{embedding_name}_{model_name}"
    model.save(filename)
    return filename

In [5]:
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print(tf.config.list_physical_devices())

Tensorflow version: 2.12.0
Keras Version: 2.12.0
GPU is NOT AVAILABLE
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## TFID Vectorization

Word Embeding layer convert text to numeric form which is used as the first layer for the deep learning model
- https://speakai.co/the-best-pretrained-word-embeddings/#:~:text=The%20most%20popular%20pretrained%20word,GloVe%2C%20Word2Vec%2C%20and%20FastText.

## Glove embedding

In [5]:
def glove_em(x_train):
    embedding_name = "glove"
    text_length = 50 #pad/truncate text to this long, such that each text after token will be this long

    custom_encoder = TextVectorization(
        standardize = None,
        output_sequence_length=text_length, 
    )
    custom_encoder.adapt(x_train)
    vocab = custom_encoder.get_vocabulary()
    print(f"total vocab {len(vocab)}")
    vocab_dict = dict(zip(vocab, range(len(vocab))))

    # load glove to dictionay
    embeddings_dic = dict()
    glove_file = open("Dataset/glove_embedding.txt", encoding="utf8")

    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dic[word] = vector_dimensions
    glove_file.close()
    print("Total words ", len(embeddings_dic))

    # create vocab length is the size of token in dictionary
    # Size of the vocabulary
    vocab_length = len(vocab) + 1
    embedding_dim = 100 #each glove word is 100 long

    hits = 0
    miss = 0

    # create embedding matrix having 100 col
    # for all vocab word we give it a vector value from glove
    # for those not found in glove will be empty 0
    # size of embedding_matriz = size of word_tokenizer.word_index.items()
    # embedding_matrix is the weight 
    embedding_matrix = np.zeros((vocab_length, embedding_dim))
    for word, index in vocab_dict.items():
        embedding_vector = embeddings_dic.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            hits += 1
        else:
            miss +=1
    print("Converted %d words (%d misses)" % (hits, miss))

    custom_embedding = Embedding(vocab_length, embedding_dim, 
                embeddings_initializer=keras.initializers.Constant(embedding_matrix), 
                trainable = False,
                input_length=text_length,
                mask_zero=True)
    
    return custom_encoder, custom_embedding, embedding_name

## FastText

- https://medium.com/@93Kryptonian/word-embedding-using-fasttext-62beb0209db9
- It treats each word as composed of n-grams. In word2vec each word is represented as a bag of words but in FastText each word is represented as a bag of character n-gram.


In [27]:
def fasttext_em(x_train):
    embedding_name = "fasttext"
    text_length = 50 #pad/truncate text to this long, such that each text after token will be this long
    vector_size=100

    sentences = [sentence.split() for sentence in x_train]
    ft_model = FastText(sentences, vector_size=vector_size, window=5, min_count=2, workers=4, seed=42, sg=1, epochs=10) # skip gram or cbow=0
    ft = ft_model.wv
    ft_vocab = ft.index_to_key

    custom_encoder = TextVectorization(
        standardize = None,
        output_sequence_length=text_length, 
        vocabulary = ft_vocab
    )

    vocab = custom_encoder.get_vocabulary()
    print(f"total vocab {len(vocab)}")
    vocab_dict = dict(zip(vocab, range(len(vocab))))

    vocab_length = len(vocab) + 1
    embedding_dim = vector_size

    hits = 0
    miss = 0

    embedding_matrix = np.zeros((vocab_length, embedding_dim))
    for word, index in vocab_dict.items():
        embedding_vector = ft[word]
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            hits += 1
        else:
            miss +=1
    print("Converted %d words (%d misses)" % (hits, miss))

    custom_embedding = Embedding(vocab_length, embedding_dim, 
                embeddings_initializer=keras.initializers.Constant(embedding_matrix), 
                trainable = False,
                input_length=text_length,
                mask_zero=True)
    return custom_encoder, custom_embedding, embedding_name

## Word2Vec

- using pretrained??

In [15]:
def word2vec_em(x_train):
    embedding_name = "word2vec"
    text_length = 50 #pad/truncate text to this long, such that each text after token will be this long
    vector_size=100

    sentences = [sentence.split() for sentence in x_train]
    w_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4, sg=1, epochs=10)
    w = w_model.wv
    w_vocab = w.index_to_key

    custom_encoder = TextVectorization(
        standardize = None,
        output_sequence_length=text_length, 
        vocabulary = w_vocab
    )

    vocab = custom_encoder.get_vocabulary()
    print(f"total vocab {len(vocab)}")
    vocab_dict = dict(zip(vocab, range(len(vocab))))

    vocab_length = len(vocab) + 1
    embedding_dim = vector_size

    hits = 0
    miss = 0

    embedding_matrix = np.zeros((vocab_length, embedding_dim))
    for word, index in vocab_dict.items():
        if word in w.key_to_index:
            embedding_vector = w[word]
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
                hits += 1
            else:
                print(word)
                miss +=1
        else:
            print(word)
            miss +=1
    print("Converted %d words (%d misses)" % (hits, miss))

    custom_embedding = Embedding(vocab_length, embedding_dim, 
                embeddings_initializer=keras.initializers.Constant(embedding_matrix), 
                trainable = False,
                input_length=text_length,
                mask_zero=True)
    return custom_encoder, custom_embedding, embedding_name

## test without pre-trained
- https://www.tensorflow.org/text/guide/word_embeddings

In [41]:
def noTrained_em(x_train):
    embedding_name = "no_train"
    text_length = 50 #pad/truncate text to this long, such that each text after token will be this long
    vector_size=100

    custom_encoder = TextVectorization(
        standardize = None,
        output_sequence_length=text_length, 
    )
    custom_encoder.adapt(x_train)
    vocab = custom_encoder.get_vocabulary()
    print(f"total vocab {len(vocab)}")
    vocab_dict = dict(zip(vocab, range(len(vocab))))

    vocab_length = len(vocab) + 1
    embedding_dim = vector_size

    custom_embedding = Embedding(vocab_length, embedding_dim,
                input_length=text_length,
                mask_zero=True)
    return custom_encoder, custom_embedding, embedding_name

## Simple Neural network

In [6]:
def snn(custom_encoder, custom_embedding, embedding_name):
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    model_name = "snn-"+embedding_name
    save_model_nn(model, model_name)
    print(history.history['val_accuracy'])

## CNN 

- usually for 2d image recognition 
- use 1d cnn for text classification

In [18]:
def cnn(custom_encoder, custom_embedding, embedding_name):
    print("cnn")
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    save_model_nn(model, "cnn", embedding_name, dataset_name)
    print(f"acc {history.history['val_accuracy'][0]}")
    return history

## RNN - LSTM

- LSTM is a variant of RNN
- input -> textvectorization (encoder/token) -> emedding -> nn layer
- Now that all samples have a uniform length, the model must be informed that some part of the data is actually padding and should be ignored. That mechanism is masking.


- sigmoid - 6s 32ms/step - loss: 0.1901 - accuracy: 0.9281 - precision: 0.9117 - recall: 0.9186 - val_loss: 0.7441 - val_accuracy: 0.7256 - val_precision: 0.6628 - val_recall: 0.7021
- relu - 7s 34ms/step - loss: 0.5181 - accuracy: 0.7423 - precision: 0.7250 - recall: 0.6280 - val_loss: 0.5530 - val_accuracy: 0.7338 - val_precision: 0.7074 - val_recall: 0.6215
- sigmoid is better for binary text classification

- dropout - 19s 98ms/step - loss: 0.3879 - accuracy: 0.8246 - precision: 0.7934 - recall: 0.7904 - val_loss: 0.5436 - val_accuracy: 0.7323 - val_precision: 0.6497 - val_recall: 0.7828

- GOOD = https://medium.com/mlearning-ai/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad

In [19]:
# this is better with the dropout
def lstm(custom_encoder, custom_embedding, embedding_name):
    print("lstm")
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    save_model_nn(model, "lstm", embedding_name, dataset_name)
    print(f"acc {history.history['val_accuracy'][0]}")
    return history

## RNN - BILSTM

In [20]:
def bilstm(custom_encoder, custom_embedding, embedding_name):
    print("bilstm")
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(Bidirectional(LSTM(128)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    save_model_nn(model, "bilstm", embedding_name, dataset_name)
    print(f"acc {history.history['val_accuracy'][0]}")
    return history

## RNN - GRU

In [21]:
def gru(custom_encoder, custom_embedding, embedding_name):
    print("gru")
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(SpatialDropout1D(0.2))
    model.add(GRU(128))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    save_model_nn(model, "gru", embedding_name, dataset_name)
    print(f"acc {history.history['val_accuracy'][0]}")
    return history

## RNN 

- https://www.tensorflow.org/text/tutorials/text_classification_rnn
- 'input_dim' = the vocab size that we will choose. In other words it is the number of unique words in the vocab.
- 'output_dim' = the number of dimensions we wish to embed into. Each word will be represented by a vector of this much dimensions.

In [22]:
def rnn(custom_encoder, custom_embedding, embedding_name):
    print("rnn")
    model = Sequential()
    model.add(custom_encoder)
    model.add(custom_embedding)
    model.add(SimpleRNN(128))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=METRICS)

    history = model.fit(x_train, y_train_binary, epochs=10,
                        validation_data=(x_test,y_test_binary))

    save_model_nn(model, "rnn", embedding_name, dataset_name)
    print(f"acc {history.history['val_accuracy'][0]}")
    return history

## Pre trained transformer
- gpt word embedding and bert
- based on transformer architecture
- uses deep learning for word embedding 
- Yes, transformer-based word embeddings are a form of deep learning. The transformer model architecture, which is the foundation of transformer-based word embeddings, is a deep learning architecture widely used in natural language processing (NLP) tasks.
- transformer-based word embeddings are a type of deep learning technique that utilizes the power of deep neural networks to learn contextually rich representations of words or tokens in natural language text.

## GPT

- GPT-2 belongs to a family of deep learning models called “Transformers”. Transformers are the building block of the current state-of-the-art NLP architecture
- A typical transformers design contains two parts, encoder and decoders, both working as vectorized representation of word relationships.
- https://github.com/openai/openai-cookbook/blob/main/examples/Fine-tuned_classification.ipynb
- can do through fine tunning or word embedding 
- 

In [2]:
import openai
import os

### Fine tuining 

- https://platform.openai.com/docs/api-reference/fine-tunes

In [112]:
# dataset for gpt
df_gpt = pd.DataFrame(zip(x_train,y_train_binary), columns = ['prompt', 'completion'])
df_gpt.to_json("Dataset/SE2019/gpt_data_train.jsonl", orient='records', lines=True)

df_gpt = pd.DataFrame(zip(x_test,y_test_binary), columns = ['prompt', 'completion'])
df_gpt.to_json("Dataset/SE2019/gpt_data_test.jsonl", orient='records', lines=True)

df_gpt["completion"].value_counts()

In [118]:
# prepare dataset for fine tune do in cmd
# !openai tools fine_tunes.prepare_data -f Dataset/SE2019/gpt_data_test.jsonl 

^C


In [14]:
# upload file to openai and create fine tune model
# openai.File.create(
#   file=open("Dataset/SE2019/gpt_data_train_prepared_train.jsonl", "rb"),
#   purpose='fine-tune'
# )

# openai.File.create(
#   file=open("Dataset/SE2019/gpt_data_train_prepared_valid.jsonl", "rb"),
#   purpose='fine-tune'
# )

# openai.FineTune.create(training_file ="file-XCtTOLLxI6jM59TlrDoT4LGL",
#                        validation_file="file-Do46v5kndAoPqdaR2ZVwbPT5",
#                        model = "ada",
#                        compute_classification_metrics = True,
#                        classification_positive_class = " Hate"
#                        )

<File file id=file-XCtTOLLxI6jM59TlrDoT4LGL at 0x1d8e3f835f0> JSON: {
  "object": "file",
  "id": "file-XCtTOLLxI6jM59TlrDoT4LGL",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 656131,
  "created_at": 1687786717,
  "status": "uploaded",
  "status_details": null
}

In [7]:
openai.FineTune.list()

<OpenAIObject list at 0x27457b19310> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-VwKO2vWcNtBJrPoZorUtFTVV",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": 8,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": 0.1,
        "classification_positive_class": " Hate",
        "compute_classification_metrics": true
      },
      "organization_id": "org-57m2RCBaIU5pd9nsTDikjeLg",
      "model": "ada",
      "training_files": [
        {
          "object": "file",
          "id": "file-XCtTOLLxI6jM59TlrDoT4LGL",
          "purpose": "fine-tune",
          "filename": "file",
          "bytes": 656131,
          "created_at": 1687786717,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [
        {
          "object": "file",
          "id": "file-Do46v5kndAoPqdaR2ZVwbPT5",
          "purpose": "fine-tune",
          "filename": "file",
        

In [119]:
# create fine tune model
# !openai api fine_tunes.create -m ada -t "Dataset/SE2019/gpt_data_train_prepared.jsonl" -v "Dataset/SE2019/gpt_data_test_prepared.jsonl" --compute_classification_metrics --classification_positive_class " 0"

^C


In [9]:
openai.FineTune.retrieve(id="ft-XGDLqYtbnvM9gy4Mrm8AWGay")

<FineTune fine-tune id=ft-XGDLqYtbnvM9gy4Mrm8AWGay at 0x27457b18ef0> JSON: {
  "object": "fine-tune",
  "id": "ft-XGDLqYtbnvM9gy4Mrm8AWGay",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null,
    "classification_positive_class": " Hate",
    "compute_classification_metrics": true
  },
  "organization_id": "org-57m2RCBaIU5pd9nsTDikjeLg",
  "model": "ada",
  "training_files": [
    {
      "object": "file",
      "id": "file-XCtTOLLxI6jM59TlrDoT4LGL",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 656131,
      "created_at": 1687786717,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-Do46v5kndAoPqdaR2ZVwbPT5",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 122361,
      "created_at": 1687786754,
      "status": "processed",
      "status_details": null
    }
  ],
  "res

In [189]:
# get result of model
#!openai api fine_tunes.results -i ft-LAcA6GtvgJ0lksqRT8jwtzSx > result.csv

results = pd.read_csv('result.csv')
results[results['classification/accuracy'].notnull()].tail(1)

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/precision,classification/recall,classification/auroc,classification/auprc,classification/f1.0
3137,3138,1017808,25104,0.030316,1.0,1.0,NaN,NaN,NaN,0.814333,0.837239,0.844728,0.890362,0.914601,0.840967


#### Predict

In [127]:
def gpt_complete_create(prompt_text):
    model_id = "ada:ft-personal-2023-06-26-17-27-28" 
    result_gpt = openai.Completion.create(model=model_id, prompt=prompt_text, max_tokens=1, temperature=0)
    return result_gpt

In [27]:
filepath = "Dataset/SE2019/gpt_data_test_prepared.jsonl"
df_gpt = pd.read_json(filepath, lines= True)
df_gpt['predicted'] = -1
df_gpt.head()

,prompt,completion
0,hey people already long islandanother chopped...,1
1,mandy manning one world reliefs friends partne...,0
2,tell us u rape people pussy ->,0
3,three streets berlin shut fighting migrants tu...,1
4,maybe visits school lebron built build one imm...,0


In [144]:
def gpt_predict(start = 0, step = 10):
    max_s = len(df_gpt)
    prompts_tosend = []
    for i in range(start, start+step):
        if i == max_s: break
        p = df_gpt.loc[i]['prompt']
        prompts_tosend.append(p)
    
    predict_result = gpt_complete_create(prompts_tosend)
    choices_gpt = predict_result.to_dict()['choices']

    for i in range(len(choices_gpt)):
        j = choices_gpt[i].to_dict()
        df_gpt.loc[start+j["index"], 'predicted'] = int(j['text'])

    print(f"predicted {start} to {start+len(prompts_tosend)-1}")

    # for i in range(start, start+step):
    #     r = df_gpt.loc[i]
    #     print(r['prompt'], "\n", r['completion'], r['predicted'])

    return prompts_tosend, predict_result


In [142]:
len(df_gpt)

2693

In [145]:
def loop_gpt(start, end, step):
    for i in range(start, end, step):
        bb, cc = gpt_predict(i, step)
loop_gpt(0, len(df_gpt), 50)

next start 2060
next start 2110
next start 2160
next start 2210
next start 2260
next start 2310
next start 2360
next start 2410
next start 2460
next start 2510
next start 2560
next start 2610
next start 2660
next start 2693


In [147]:
df_gpt.loc[df_gpt['predicted'] == -1]

,prompt,completion,predicted


In [152]:
y_test = df_gpt['completion'].to_numpy()
y_test_pred = df_gpt['predicted'].to_numpy()

y_test = np.where(y_test == 1, "Hate", "Non-Hate") 
y_test_pred = np.where(y_test_pred == 1, "Hate", "Non-Hate") 

In [157]:
df_gpt.to_csv("Dataset/SE2019/gpt_data_test_result", index=False)

In [158]:
print(classification_report(y_test, y_test_pred, labels=["Hate","Non-Hate"]))

              precision    recall  f1-score   support

        Hate       0.78      0.78      0.78      1128
    Non-Hate       0.84      0.84      0.84      1565

    accuracy                           0.81      2693
   macro avg       0.81      0.81      0.81      2693
weighted avg       0.81      0.81      0.81      2693



## BERT

- https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270
- https://www.youtube.com/watch?v=hOCDJyZ6quA
- tensorflow hub bert https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4
- bert will convert sentence into embeding vector which will feed to neural network for training 
- consist of preprocess and embeding 
- (4)BERT-RNN: The corresponding representational word vectors were trained by BERT model for the input text, which were then classified by RNN neural network. (5)word2vec-RNN: This model is a traditional text classification model. 4.3.
- BERT is a neural-network-based technique for language processing pre-training
- it is not a classification algorithm 
- BERT generates <b>contextual embeddings</b>, the input to the model is a sentence rather than a single word.
- BERT learns contextualized word representations, often referred to as contextual word embeddings or contextualized embeddings. Unlike traditional word embeddings, which assign a fixed vector representation to each word, BERT's word representations are sensitive to the context in which the word appears.

## T5

## Summary

In [51]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 50, 100)           3200300   
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 50, 100)          0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                

## Predict

In [15]:
def nn_predict():
    score = model.evaluate(x_test, y_test_binary, verbose=0)
    print("Score: ", score[0])
    print("Accuracy: ", score[1])

    y_test_pred_percent = model.predict(x_test, verbose=0)
    y_test_pred = np.where(y_test_pred_percent > 0.5, "Hate", "Non-Hate") 
    y_test_pred = y_test_pred.flatten()

    return y_test_pred

## Visualise

In [23]:
def plot_history(model_history):
    # Model performance charts
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)

    plt.plot(model_history.history['accuracy'])
    plt.plot(model_history.history['val_accuracy'])

    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.ylim(None, 1)
    plt.subplot(1, 2, 2)

    plt.plot(model_history.history['loss'])
    plt.plot(model_history.history['val_loss'])

    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.ylim(0, None)


# Results

In [18]:
def plot_heatmap(y_test, y_test_pred):
    # Heatmap
    ax = plt.subplot()

    # Plot the two-way Confusion Matrix
    sb.heatmap(confusion_matrix(y_test, y_test_pred, labels=["Hate","Non-Hate"]), 
            annot = True, fmt=".0f", annot_kws={"size": 18}, ax=ax)

    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.xaxis.set_ticklabels(["Hate","Non-Hate"])
    ax.yaxis.set_ticklabels(["Hate","Non-Hate"])

    # Count
    df1 = pd.DataFrame({'Actual':y_test, 'Predict':y_test_pred})
    # print(df1.describe())
    print(f"Count: {df1['Actual'].value_counts()}")
    print()
    print(f"Count: {df1['Predict'].value_counts()}")
    print()

# Model compare

In [6]:
# for i in model_to_load:
#     filename = f"models/{i}.pickle"
#     old_model = pickle.load(open(filename,"rb"))

def get_classification_report(i, cr):
    return [i, cr['accuracy'], cr['macro avg']['precision'], 
            cr['macro avg']['recall'], cr['macro avg']['f1-score'],
            cr['Hate']['f1-score'],cr['Non-Hate']['f1-score'], 
            cr['Hate']['support'],cr['Non-Hate']['support']]


def get_result_nn(model_to_load):
    c = ['Model', 'Accuracy', 'precision', 'recall', 'f1-score', 'hate f1', "non-hate f1", 'hate support', 'non-hate support']
    result_table = pd.DataFrame(columns=c)
    for i in model_to_load:
        filename = f"models/{i}"
        print(filename)
        old_model = load_model(filename)

        y_test_pred = old_model.predict(x_test, verbose=0)
        y_test_pred = np.where(y_test_pred > 0.5, "Hate", "Non-Hate") 
        y_test_pred = y_test_pred.flatten()

        cr = classification_report(y_test, y_test_pred, labels=["Hate","Non-Hate"], output_dict=True)
        result_table.loc[len(result_table)] = get_classification_report(i, cr)
    return result_table.style.highlight_max(color = 'red', axis = 0)

# test

In [36]:
x_train[0]

'anyone help older samsung tablet get white screen trying log gab download says parsing error bookmarked still wont open even online browser help'

In [13]:
print(len(x_train), len(x_test))

18979 8134


In [42]:
custom_encoder, custom_embedding, embedding_name = noTrained_em(x_train)
print(embedding_name)
print(dataset_name)

total vocab 37041
no_train
GabHateCorpus


0.30 fold
- se2019 = Converted 7613 words (2 misses)
- gab = Converted 13974 words (2 misses)

10 fold
- gab, word2vec = Converted 16230 words (2 misses)



In [43]:
h = cnn(custom_encoder,custom_embedding,embedding_name)
h = rnn(custom_encoder,custom_embedding,embedding_name)
h = lstm(custom_encoder,custom_embedding,embedding_name)
# h = bilstm(custom_encoder,custom_embedding,embedding_name)
h = gru(custom_encoder,custom_embedding,embedding_name)

cnn
Epoch 1/10
763/763 [==============================] - 35s 44ms/step - loss: 0.3068 - accuracy: 0.8880 - precision: 0.6301 - recall: 0.2231 - val_loss: 0.2695 - val_accuracy: 0.8931 - val_precision: 0.6594 - val_recall: 0.2725
Epoch 2/10
763/763 [==============================] - 31s 41ms/step - loss: 0.1754 - accuracy: 0.9336 - precision: 0.8073 - recall: 0.6066 - val_loss: 0.3079 - val_accuracy: 0.8879 - val_precision: 0.5798 - val_recall: 0.3263
Epoch 3/10
763/763 [==============================] - 33s 43ms/step - loss: 0.0583 - accuracy: 0.9812 - precision: 0.9525 - recall: 0.8920 - val_loss: 0.4039 - val_accuracy: 0.8831 - val_precision: 0.5392 - val_recall: 0.3503
Epoch 4/10
763/763 [==============================] - 31s 41ms/step - loss: 0.0180 - accuracy: 0.9947 - precision: 0.9885 - recall: 0.9684 - val_loss: 0.4873 - val_accuracy: 0.8779 - val_precision: 0.5059 - val_recall: 0.3862
Epoch 5/10
763/763 [==============================] - 31s 41ms/step - loss: 0.0082 - accurac

INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_cnn\assets


INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_cnn\assets


acc 0.8930678367614746
rnn
Epoch 1/10
763/763 [==============================] - 115s 148ms/step - loss: 0.1489 - accuracy: 0.9422 - precision: 0.8159 - recall: 0.6863 - val_loss: 0.4125 - val_accuracy: 0.8698 - val_precision: 0.4570 - val_recall: 0.3024
Epoch 2/10
763/763 [==============================] - 121s 158ms/step - loss: 0.0597 - accuracy: 0.9793 - precision: 0.9230 - recall: 0.9083 - val_loss: 0.4803 - val_accuracy: 0.8551 - val_precision: 0.4013 - val_recall: 0.3593
Epoch 3/10
763/763 [==============================] - 119s 156ms/step - loss: 0.0302 - accuracy: 0.9889 - precision: 0.9576 - recall: 0.9525 - val_loss: 0.6529 - val_accuracy: 0.8654 - val_precision: 0.4317 - val_recall: 0.2934
Epoch 4/10
763/763 [==============================] - 117s 154ms/step - loss: 0.0173 - accuracy: 0.9945 - precision: 0.9813 - recall: 0.9744 - val_loss: 0.7736 - val_accuracy: 0.8595 - val_precision: 0.4041 - val_recall: 0.2964
Epoch 5/10
763/763 [==============================] - 115s 15

INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_rnn\assets


...

INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_bilstm\assets


acc 0.844763994216919
gru
Epoch 1/10
763/763 [==============================] - 216s 278ms/step - loss: 0.0371 - accuracy: 0.9689 - precision: 0.8759 - recall: 0.8717 - val_loss: 0.8563 - val_accuracy: 0.8378 - val_precision: 0.3209 - val_recall: 0.2844
Epoch 2/10
763/763 [==============================] - 199s 261ms/step - loss: 0.0039 - accuracy: 0.9990 - precision: 0.9983 - recall: 0.9937 - val_loss: 1.0127 - val_accuracy: 0.8355 - val_precision: 0.3427 - val_recall: 0.3653
Epoch 3/10
763/763 [==============================] - 201s 264ms/step - loss: 0.0038 - accuracy: 0.9990 - precision: 0.9970 - recall: 0.9947 - val_loss: 0.9929 - val_accuracy: 0.8348 - val_precision: 0.3443 - val_recall: 0.3772
Epoch 4/10
763/763 [==============================] - 204s 268ms/step - loss: 0.0040 - accuracy: 0.9988 - precision: 0.9963 - recall: 0.9937 - val_loss: 1.0385 - val_accuracy: 0.8274 - val_precision: 0.3255 - val_recall: 0.3743
Epoch 5/10
763/763 [==============================] - 205s 269

INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_gru\assets


INFO:tensorflow:Assets written to: models/GabHateCorpus_no_train_gru\assets


acc 0.8377581238746643


In [51]:
ds ="Implicit_hate_corpus"
em = 'fasttext'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

models/Implicit_hate_corpus_fasttext_cnn
models/Implicit_hate_corpus_fasttext_rnn
models/Implicit_hate_corpus_fasttext_lstm
models/Implicit_hate_corpus_fasttext_bilstm
models/Implicit_hate_corpus_fasttext_gru


,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,Implicit_hate_corpus_fasttext_cnn,0.691962,0.672204,0.673499,0.672811,0.593654,0.751968,2419,4025
1,Implicit_hate_corpus_fasttext_rnn,0.699876,0.682861,0.647670,0.652007,0.522940,0.781073,2419,4025
2,Implicit_hate_corpus_fasttext_lstm,0.714618,0.698594,0.669451,0.675276,0.562247,0.788304,2419,4025
3,Implicit_hate_corpus_fasttext_bilstm,0.717722,0.700199,0.677956,0.683571,0.579616,0.787525,2419,4025
4,Implicit_hate_corpus_fasttext_gru,0.714618,0.703246,0.662193,0.667807,0.543106,0.792508,2419,4025


In [44]:
ds ="GabHateCorpus"
em = 'no_train'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

models/GabHateCorpus_no_train_cnn
models/GabHateCorpus_no_train_rnn
models/GabHateCorpus_no_train_lstm
models/GabHateCorpus_no_train_bilstm
models/GabHateCorpus_no_train_gru


,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,GabHateCorpus_no_train_cnn,0.879794,0.714788,0.645798,0.670194,0.407273,0.933114,334,2378
1,GabHateCorpus_no_train_rnn,0.832596,0.608088,0.604730,0.606354,0.307927,0.904782,334,2378
2,GabHateCorpus_no_train_lstm,0.842552,0.629033,0.621988,0.625322,0.340031,0.910613,334,2378
3,GabHateCorpus_no_train_bilstm,0.802729,0.608090,0.650749,0.621349,0.359281,0.883417,334,2378
4,GabHateCorpus_no_train_gru,0.835914,0.625620,0.631070,0.628232,0.350365,0.906098,334,2378


In [32]:
ds ="GabHateCorpus"
em = 'word2vec'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

models/GabHateCorpus_word2vec_cnn
models/GabHateCorpus_word2vec_rnn
models/GabHateCorpus_word2vec_lstm
models/GabHateCorpus_word2vec_bilstm
models/GabHateCorpus_word2vec_gru


,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,GabHateCorpus_word2vec_cnn,0.887906,0.746150,0.642703,0.674457,0.410853,0.938060,334,2378
1,GabHateCorpus_word2vec_rnn,0.873894,0.694916,0.633426,0.655119,0.380435,0.929803,334,2378
2,GabHateCorpus_word2vec_lstm,0.888274,0.775411,0.591443,0.621357,0.303448,0.939266,334,2378
3,GabHateCorpus_word2vec_bilstm,0.884956,0.734727,0.633300,0.663554,0.390625,0.936482,334,2378
4,GabHateCorpus_word2vec_gru,0.893437,0.807469,0.605968,0.641856,0.341686,0.942026,334,2378


In [40]:
ds ="GabHateCorpus"
em = 'fasttext'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

,Model,Accuracy,precision,recall,f1-score,hate f1,hate support,non-hate f1,non-hate support
0,GabHateCorpus_fasttext_cnn,0.883944,0.724345,0.603634,0.632063,0.327635,983,0.936491,7151
1,GabHateCorpus_fasttext_rnn,0.879149,0.689995,0.503949,0.476797,0.017982,983,0.935613,7151
2,GabHateCorpus_fasttext_lstm,0.891320,0.795910,0.584577,0.613565,0.285945,983,0.941184,7151
3,GabHateCorpus_fasttext_bilstm,0.890214,0.755402,0.620362,0.653948,0.368011,983,0.939886,7151
4,GabHateCorpus_fasttext_gru,0.889107,0.740086,0.664044,0.691493,0.444581,983,0.938405,7151


In [38]:
ds = 'GabHateCorpus'
em = 'glove'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

,Model,Accuracy,Precision,Recall,F1-Score
0,GabHateCorpus_glove_cnn,0.887755,0.864472,0.887755,0.864063
1,GabHateCorpus_glove_rnn,0.885050,0.858332,0.885050,0.852100
2,GabHateCorpus_glove_lstm,0.892304,0.880023,0.892304,0.884057
3,GabHateCorpus_glove_bilstm,0.880747,0.872886,0.880747,0.876315
4,GabHateCorpus_glove_gru,0.882837,0.876953,0.882837,0.879606


In [7]:
ds = 'SE2019'
em = 'word2vec'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

models/SE2019_word2vec_cnn
models/SE2019_word2vec_rnn
models/SE2019_word2vec_lstm
models/SE2019_word2vec_bilstm
models/SE2019_word2vec_gru


,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,SE2019_word2vec_cnn,0.700488,0.696548,0.700337,0.697045,0.664750,0.729341,1653,2240
1,SE2019_word2vec_rnn,0.669407,0.660556,0.655255,0.656664,0.590519,0.722809,1653,2240
2,SE2019_word2vec_lstm,0.713332,0.711090,0.693266,0.696179,0.623989,0.768369,1653,2240
3,SE2019_word2vec_bilstm,0.713075,0.708568,0.695976,0.698712,0.632928,0.764495,1653,2240
4,SE2019_word2vec_gru,0.717185,0.710524,0.706840,0.708230,0.657116,0.759344,1653,2240


In [36]:
ds = 'SE2019'
em = 'fasttext'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

models/SE2019_fasttext_cnn
models/SE2019_fasttext_rnn
models/SE2019_fasttext_lstm
models/SE2019_fasttext_bilstm
models/SE2019_fasttext_gru


,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,SE2019_fasttext_cnn,0.701259,0.696076,0.699025,0.696878,0.660438,0.733318,1653,2240
1,SE2019_fasttext_rnn,0.682764,0.679793,0.658064,0.659184,0.569536,0.748831,1653,2240
2,SE2019_fasttext_lstm,0.704598,0.697699,0.691701,0.693555,0.635384,0.751727,1653,2240
3,SE2019_fasttext_bilstm,0.711790,0.705390,0.706433,0.705855,0.664072,0.747638,1653,2240
4,SE2019_fasttext_gru,0.708965,0.702250,0.702471,0.702358,0.658014,0.746702,1653,2240


In [45]:
ds = 'SE2019'
em = 'glove'
get_result_nn([ds+"_"+em+"_cnn", ds+"_"+em+"_rnn", ds+"_"+em+"_lstm",ds+"_"+em+"_bilstm",ds+"_"+em+"_gru"])

,Model,Accuracy,Precision,Recall,F1-Score
0,SE2019_glove_cnn,0.709989,0.707252,0.709989,0.707052
1,SE2019_glove_rnn,0.661344,0.661685,0.661344,0.661509
2,SE2019_glove_lstm,0.732269,0.746559,0.732269,0.734036
3,SE2019_glove_bilstm,0.726328,0.730623,0.726328,0.727574
4,SE2019_glove_gru,0.756034,0.754855,0.756034,0.755169


In [149]:
get_result()

,Model,Accuracy,Precision,Recall,F1-Score
0,dtc,0.749350,0.747932,0.749350,0.748249
1,dtc-tfid,0.743780,0.743531,0.743780,0.743649
2,dtc-w2v,0.586706,0.589898,0.586706,0.588024
3,svm,0.768659,0.767394,0.768659,0.766195
4,svm-tfid,0.746008,0.760112,0.746008,0.731861
5,svm-w2v,0.590048,0.611645,0.590048,0.461541
6,lr,0.684738,0.690064,0.684738,0.686326
7,lr-tfid,0.694393,0.697312,0.694393,0.695449
8,lr-w2v,0.708504,0.705758,0.708504,0.705809
